### 📓👋 Welcome to the industries evaluation notebook.

The key aim of the notebook is to evaluate the performance of the SICMapper and the IndustryMeasures overall. It is split into the following sections:

**0️⃣ Null analysis:** On the full dataset of extracted industries measures, what proportion are null? This includes null analysis of SIC codes in addition to industries measures. 

**🌊 Threshold analysis:** On the full dataset of extracted industries measures, what is the optimal threshold for the two SIC mapping approaches? This includes a thresholding analysis of the `closest SIC` and `majority SIC` approaches to better understand completeness.

**🤔 Labelled Evaluation:** This section is split into two subsections: 
- one that explores a labelled dataset of job ads to explore the effect of different thresholds on accuracy. 
- one that explores a different, labelled dataset of job ads (incl. SIC codes mapped via companies house) to explore overall accuracy. 


In [611]:
from dap_prinz_green_jobs.getters.data_getters import load_s3_data

from dap_prinz_green_jobs import BUCKET_NAME, PROJECT_DIR

import pandas as pd
import altair as alt
import numpy as np
import os 

from datetime import datetime

In [612]:
# global variables, settings, create directory for the whole notebook

alt.data_transformers.disable_max_rows()

today = datetime.today().strftime('%y%m%d')
graph_dir = str(PROJECT_DIR / f"outputs/figures/evaluation/industries/{today}/")

if not os.path.exists(graph_dir):
    print(f"Creating {graph_dir} directory")
    os.makedirs(graph_dir)
else:
    print(f"{graph_dir} directory already exists")

/Users/india.kerlenesta/Projects/dap_green_jobs/dap_prinz_green_jobs/outputs/figures/evaluation/industries/231011 directory already exists


### 💾 0. Load data
- full dataset of SIC codes
- evaluation dataset of labelled SIC mappings

In [613]:
print('loading full dataset of SIC codes...')
production = "True"
config="base"
date_stamp = "20231007"

green_inds_outputs = load_s3_data(
        BUCKET_NAME,
        f"outputs/data/ojo_application/extracted_green_measures/{date_stamp}/ojo_sample_industry_green_measures_production_{production}_{config}.json",
    )

inds_measures_df = pd.DataFrame.from_dict(green_inds_outputs, orient='index').reset_index().rename(columns={'index':'id'})

print('loading labelled threshold evaluation dataset...')
threshold_evaluation_df = pd.read_csv('s3://prinz-green-jobs/outputs/data/labelled_job_adverts/evaluation/industries/231010_threshold_eval_df_ojo_sicmapper_labelled.csv')

print('loading overall labelled evaluation dataset...')
labelled_evaluation_df = pd.read_csv('s3://prinz-green-jobs/outputs/data/labelled_job_adverts/evaluation/industries/231010_evaluation_df_labelled.csv')

loading full dataset of SIC codes...
loading labelled threshold evaluation dataset...
loading overall labelled evaluation dataset...


### 0️⃣ 1. Null analysis

This section explores the null values in the dataset (incl. industry measures) without doing any additional thresholding.

In [614]:
#% of SIC codes missing
inds_measures_df_nona = inds_measures_df[inds_measures_df['SIC'].notna()]

sic_na_df = pd.DataFrame({'no': inds_measures_df['SIC'].isna().sum(), 
                          'yes': inds_measures_df_nona.shape[0]}, index=[0]).T.rename(columns={0:'count'}).reset_index().rename(columns={'index':'SIC'})

#make bar chart all add up to 100%
sic_count_bar = (alt.Chart(sic_na_df).mark_bar().encode(
    y=alt.Y('SIC', sort='-y', title='SIC code present'),
    #add light blue to bars to add up to the full count of job ads
    x=alt.X('count', title="Number of job ads"),
    color=alt.Color('SIC', legend=None))).properties(
    title={
      "text": ["Percentage (%) of SIC codes missing"], 
      "subtitle": [f"{round(((len(inds_measures_df_nona)/len(inds_measures_df))*100),2)}% of job adverts have a SIC code associated to them.", ""],
      "color": "black",
      "subtitleColor": "black"
    }
)

# SIC code matching method
sic_method_count = (inds_measures_df_nona
                    ['SIC_method']
                    .value_counts()
                    .reset_index()
                    .rename(columns={'index':'SIC_method','SIC_method':'count'}))


#make bar chart all add up to 100%
sic_method_count_bar = (alt.Chart(sic_method_count).mark_bar().encode(
    y=alt.Y('SIC_method', sort='y', title='Matching method'),
    #add light blue to bars to add up to the full count of job ads
    x=alt.X('count', 
            title="Number of job ads", 
            ))).properties(title="SIC matching method")

#calcualte the % of missing values per column
#drop SIC confidence column as it is not relevant for this analysis
inds_measures_df_no_conf = inds_measures_df.drop(columns=['SIC_confidence'])
na_df = (inds_measures_df_no_conf
         .isna()
         .mean()
         .round(4) * 100).reset_index().rename(columns={'index':'column',0:'percent_missing'})

missing_chart = alt.Chart(na_df).mark_bar().encode(
    x=alt.X('percent_missing', title='% Missing'),
    #change the label limit to show all columns
    y=alt.Y('column', sort='-x', title='', axis=alt.Axis(labelLimit=1000)),
    #color condition if % missing is greater than 20% then color red
    color=alt.condition(alt.datum.percent_missing > 18, alt.value('red'), alt.value('blue'))).properties(
    title={
      "text": ["Percentage (%) of missing values by column"], 
      "subtitle": ["columns in red have more than 18% in missing values", ""],
      "color": "black",
      "subtitleColor": "black"
    }
)

null_analysis_charts = ((sic_method_count_bar & sic_count_bar)| missing_chart).properties(title="NA analysis: No additional thresholds applied")

print('saving charts to html...')
null_analysis_charts.save(f"{graph_dir}/null_analysis_charts.html")
print('done!')

saving charts to html...
done!


### 🌊  2. Thresholding Analysis

The `IndustryMeasures` class maps to a SIC code based on the following default thresholds:

1. **`closest SIC`**: **0.5**
2. **`majority SIC`**: **0.3**

This section produces graphs that analyse the effect of thresholding on completeness. 

##### Generate graphs on the complete dataset to investigate the effect of thresholding on completeness

In [615]:
#drop rows where the SIC method is "companies house" i.e. there is no SIC confidence
#score associated to the SIC code
inds_measures_df_nona_noncomp = inds_measures_df_nona.query('SIC_method != "companies house"')

#plot the overall distribution of SIC confidence scores for the SIC codes
#generated by the "closest distance" method
map_type = "closest distance"
closest_dist_df = inds_measures_df_nona_noncomp.query(f'SIC_method == "{map_type}"')
closet_dist_dens = alt.Chart(closest_dist_df).transform_density(
    'SIC_confidence',
    as_=['SIC_confidence', 'density'],).mark_area().encode(
    x=alt.X("SIC_confidence:Q", title="confidence score"),
    y='density:Q',
    color=alt.value('purple'),
).properties(title=f"'{map_type}'")
    
#plot the overall distribution of SIC confidence scores for the SIC codes
#generated by the "majority" method
map_type = "majority SIC"
major_sic_df = inds_measures_df_nona_noncomp.query(f'SIC_method == "{map_type}"')
major_sic_dens = alt.Chart(major_sic_df).transform_density(
    'SIC_confidence',
    as_=['SIC_confidence', 'density'],).mark_area().encode(
    x=alt.X("SIC_confidence:Q", title="confidence score"),
    y='density:Q',
    color=alt.value('orange'),
).properties(title=f"'{map_type}'")
    
#clean up the distribution plots by adding a title and subtitle
dists = (closet_dist_dens | major_sic_dens).properties(
    title={
      "text": ["Threshold distribution by SIC matching method"], 
      "subtitle": [""],
      "color": "black",
      "subtitleColor": "black"
    }
)

#Plot the # of job ads at different minimum SIC confidence thresholds

##for the closest distance method
bins=[0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1]
cum_count = {}
for bin in bins:
    no_job_ads = closest_dist_df.query(f'SIC_confidence >= {bin}').id.nunique()
    cum_count[bin] = no_job_ads
cum_count_df = pd.DataFrame.from_dict(cum_count, orient='index').reset_index().rename(columns={'index':'SIC_confidence_range', 0:'cumulative_count'})
cum_count_df['cumulative_percent'] = round((cum_count_df['cumulative_count']/closest_dist_df.id.nunique())*100,2)

cumsum_area_chart = alt.Chart(cum_count_df, width=600).mark_area().encode(
    #start the X acis at 0.5
    x=alt.X('SIC_confidence_range', title='SIC confidence score', axis=alt.Axis(labelAngle=45), scale=alt.Scale(zero=False)),
    # Plot the calculated field created by the transformation
    y=alt.Y('cumulative_percent:Q', title='% of job ads (cumulative)'),
    color=alt.value('purple'),
)

percent_thresh_bar = alt.Chart(cum_count_df).mark_bar().encode(
    x=alt.X('cumulative_percent', title='% of job ads (cumulative)'),
    y=alt.Y('SIC_confidence_range:N', title='minimum threshold'),
    color=alt.value('purple'))

closest_dist_thresholding = (cumsum_area_chart | percent_thresh_bar).properties(title="'closest distance'")

##for the majority SIC method
bins=[0.30, 0.33, 0.36, 0.39, 0.42, 0.45]
cum_count = {}
for bin in bins:
    no_job_ads = major_sic_df.query(f'SIC_confidence >= {bin}').id.nunique()
    cum_count[bin] = no_job_ads
cum_count_df = pd.DataFrame.from_dict(cum_count, orient='index').reset_index().rename(columns={'index':'SIC_confidence_range', 0:'cumulative_count'})
cum_count_df['cumulative_percent'] = round((cum_count_df['cumulative_count']/major_sic_df.id.nunique())*100,2)

cumsum_area_chart = alt.Chart(cum_count_df, width=600).mark_area().encode(
    x=alt.X('SIC_confidence_range', title='SIC confidence score', axis=alt.Axis(labelAngle=45), scale=alt.Scale(zero=False)),
    y=alt.Y('cumulative_percent:Q', title='% of job ads (cumulative)'),
    color=alt.value('orange'),
)

percent_thresh_bar = alt.Chart(cum_count_df).mark_bar().encode(
    x=alt.X('cumulative_percent', title='% of job ads (cumulative)'),
    y=alt.Y('SIC_confidence_range:N', title='minimum threshold'),
    color=alt.value('orange'))

##combine them and add title/subtitle
maj_sic_thresholding = (cumsum_area_chart | percent_thresh_bar).properties(title="'majority SIC'")

threshold_analysis_charts = dists & (closest_dist_thresholding & maj_sic_thresholding).properties(
    title={
      "text": ["Percent (%) of job ads at different SIC confidence thresholds"], 
      "subtitle": [""],
      "color": "black",
      "subtitleColor": "black"
    }
)


print('saving charts to html...')
threshold_analysis_charts.save(f"{graph_dir}/threshold_analysis_charts.html")
print('done!')

saving charts to html...
done!


### 🤔 3. Labelled Evaluation

In this section, we evaluate **two** labelled datasets:

1. **_A labelled thresholding evaluation set_**: This labelled dataset does not include any SIC code matches to company house. It is used to evaluate the effect of thresholding on completeness and SIC quality. The dataset includes **111** job ads (`random_state=42`).

2. **_An overall evaluation set_**: This labelled dataset uses the `SicMapper` to map to a SIC code. As a result, it also includes code matches to company house. It is used to evaluate the overall performance of the `SicMapper`. The dataset includes **500** job ads (`random_state=62`). 

#### 🌊 3.1 Threshold evaluation

##### Print some high level statistics on the labelled thresholding evaluation set

In [616]:
print(f"Number of job ads in threshold evaluation set: {threshold_evaluation_df.id.nunique()}")
print(f"Number of job ads with extracted SIC: {threshold_evaluation_df[threshold_evaluation_df['match_quality[0-bad, 1-ok, 2-good]'].notnull()].id.nunique()}")
print(f"% of job ads WITH extracted SIC: {threshold_evaluation_df[threshold_evaluation_df['match_quality[0-bad, 1-ok, 2-good]'].notnull()].id.nunique()/len(threshold_evaluation_df)}")
print(f"% of job ads WITHOUT extracted SIC: {threshold_evaluation_df[threshold_evaluation_df['match_quality[0-bad, 1-ok, 2-good]'].isna()].id.nunique()/len(threshold_evaluation_df)}")

Number of job ads in threshold evaluation set: 111
Number of job ads with extracted SIC: 92
% of job ads WITH extracted SIC: 0.8288288288288288
% of job ads WITHOUT extracted SIC: 0.17117117117117117


##### Clean up labelled thresholding evaluation set

In [617]:
#clean up dataframe
threshold_evaluation_df.rename(columns={'match_quality[0-bad, 1-ok, 2-good]':'match_quality'}, inplace=True)
threshold_evaluation_df_labelled = threshold_evaluation_df[threshold_evaluation_df.match_quality.notna()]
threshold_evaluation_df_labelled['closest_dist_sic_confidence_threshold'] = pd.cut(threshold_evaluation_df_labelled['sic_confidence'],
                                                                      bins=[0.5, 0.55, 0.6, 0.65, 0.7],
                                                                    labels=['0.5-0.55', '0.55-0.6', '0.6-0.65', '0.65-0.7'])

threshold_evaluation_df_labelled['majority_sic_confidence_threshold'] = pd.cut(threshold_evaluation_df_labelled['sic_confidence'],
                                                                      bins=[0.3, 0.33, 0.36, 0.39, 0.42, 0.45],
                                                                    labels=['0.3-0.33', '0.33-0.36', '0.36-0.39', '0.39-0.42', '0.42-0.45'])

/var/folders/cq/9gxjkt2j2g1c0cfhjb2qtz000000gn/T/ipykernel_29305/1365476889.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  threshold_evaluation_df_labelled['closest_dist_sic_confidence_threshold'] = pd.cut(threshold_evaluation_df_labelled['sic_confidence'],
/var/folders/cq/9gxjkt2j2g1c0cfhjb2qtz000000gn/T/ipykernel_29305/1365476889.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  threshold_evaluation_df_labelled['majority_sic_confidence_threshold'] = pd.cut(threshold_evaluation_df_labelled['sic_conf

##### Generate graphs investigating the thresholding evaluation set

In [618]:
match_qual_df = (round(
    threshold_evaluation_df_labelled
    .match_quality
    .value_counts()/len(threshold_evaluation_df_labelled)*100, 2)
.reset_index().rename(columns={'index':'match_quality', 'match_quality':'percent'}))

match_quality = alt.Chart(match_qual_df).mark_bar().encode(
    y=alt.Y('match_quality:O', title='match quality', sort='-x'),
    x=alt.X('percent:Q', title='% of job ads'),
    color=alt.condition(alt.datum.match_quality > 0, alt.value('green'), alt.value('red'))).properties(
    title={
      "text": ["Overall Match quality"], 
      "subtitle": [f"{match_qual_df.query('match_quality > 0').percent.sum()}% of matches are ok or good", ""],
      "color": "black",
      "subtitleColor": "black"
    }
)
    
qual_by_thresh = (threshold_evaluation_df_labelled
.groupby(['closest_dist_sic_confidence_threshold', 'match_quality'])
.id
.count()
.reset_index()
.rename(columns={'id':'count'}))

qual_dict = {'1': 'ok', '2': 'good', '0': 'bad'}
qual_by_thresh['match_quality'] = qual_by_thresh['match_quality'].astype(int).astype(str)
qual_by_thresh['match_quality_label'] = qual_by_thresh['match_quality'].map(qual_dict)

#match quality by threshold

match_qual_per_range = alt.Chart(qual_by_thresh).mark_bar().encode(
    y=alt.Y('closest_dist_sic_confidence_threshold', title='SIC confidence threshold'),
    x=alt.X('count'),
    color=alt.Color('match_quality_label', title='match quality')).properties(
        title="Closest SIC")
    
#by majority SIC confidence threshold
qual_by_thresh = (threshold_evaluation_df_labelled
.groupby(['majority_sic_confidence_threshold', 'match_quality'])
.id
.count()
.reset_index()
.rename(columns={'id':'count'}))

qual_dict = {'1': 'ok', '2': 'good', '0': 'bad'}
qual_by_thresh['match_quality'] = qual_by_thresh['match_quality'].astype(int).astype(str)
qual_by_thresh['match_quality_label'] = qual_by_thresh['match_quality'].map(qual_dict)

#match quality by threshold
match_qual_per_range_maj_sic = alt.Chart(qual_by_thresh).mark_bar().encode(
    y=alt.Y('majority_sic_confidence_threshold', title='SIC confidence threshold'),
    x=alt.X('count'),
    color=alt.Color('match_quality_label', title='match quality')).properties(
        title="Majority SIC")
    
match_qual_charts = match_quality & (match_qual_per_range | match_qual_per_range_maj_sic).properties(
title={
    "text": ["Match quality by SIC confidence threshold"], 
    "subtitle": ["Thresholding slightly higher (~0.55) for the closest SIC approach could minimise bad/ok matches.", 
                "Meanwhile, there isn't a clear pattern for the majority SIC approach.", ""],
    "color": "black",
    "subtitleColor": "black"
}
)

print('saving charts to html...')
match_qual_charts.save(f"{graph_dir}/match_qual_charts.html")
print('done!')

saving charts to html...
done!


**key takeaways:**

- the best threshold for the closest SIC is likely still **0.5**
- there is no clear best threshold for the majority SIC. It appears to perform relatively worse at every confidence threshold.

#### 🖊️ 3.2 Overall evaluation: Analyse labelled dataset for overall performance

##### Clean up labelled evaluation set

In [619]:
#clean up dataset
labelled_evaluation_df = (labelled_evaluation_df
                          #drop unused columns
                          .drop(columns=['company_name_x', 'company_name_y'])
                          #rename columns to be easier to work with
                          .rename(columns={'company_description_quality[0-bad, 1-ok, 2-good]':'company_description_quality',
                                           'match_quality[0-bad, 1-ok, 2-good]': 'match_quality'})
                          #drop any rows that weren't labelled
                          .dropna(subset=['match_quality'])
                          #convert specific columns to different types, create new columns incl.
                          #length of company description and binary match quality to convert 
                          #'ok' and 'good' to 1 and 'bad' to 0
                          .assign(match_quality=lambda x: x.match_quality.astype('Int64'),
                                  company_description_quality=lambda x: x.company_description_quality.astype('Int64'),
                                  company_description_length = lambda x: x.company_description.str.len(),
                                  match_quality_binary = lambda x: np.where(x.match_quality > 0, 1, 0))
                          #remove a mistaken label
                          .query('sic_confidence != 2') 
                          .reset_index(drop=True))

##### Print some high level statistics on the evaluation set

In [620]:
#print some high level stats
print(f"{len(labelled_evaluation_df)} job ads were labelled")
print('')
print('the 10 most common SIC codes in the labelled evaluation dataset are:')
print((labelled_evaluation_df
               .sic_name
               .value_counts()
               .head(10)
               .reset_index()
               .rename(columns={'index':'sic_name', 'sic_name':'count'})
               .sic_name
               .tolist()))
print('')
print('the correlation between company description quality and match quality is:')
print(round(labelled_evaluation_df[['company_description_quality', 'match_quality']].corr()['company_description_quality'].match_quality,2))

266 job ads were labelled

the 10 most common SIC codes in the labelled evaluation dataset are:
['Management consultancy activities (other than financial management)', 'Freight air transport and space transport', 'Other business support service activities nec', 'Environmental consulting activities', 'Financial service activities, except insurance and pension funding', 'Remediation activities and other waste management services', 'Manufacture of food products', 'Management of real estate on a fee or contract basis', 'Activities of employment placement agencies', 'Civil engineering']

the correlation between company description quality and match quality is:
0.26


##### Generate graphs investigating the evaluation set

In [621]:
#generate charts 

match_qual_df = (labelled_evaluation_df.match_quality.value_counts()/len(labelled_evaluation_df)*100).reset_index().rename(columns={'index':'match_quality', 'match_quality':'percent'})

match_quality_chart = alt.Chart(match_qual_df).mark_bar().encode(
    y=alt.Y('match_quality:O', title='match quality', sort='-x'),
    x=alt.X('percent:Q', title='% of job ads'),
    color=alt.condition(alt.datum.match_quality > 0, alt.value('green'), alt.value('red'))).properties(
    title={
      "text": ["Overall Match Quality"], 
      "subtitle": [f"{round(match_qual_df.query('match_quality > 0').percent.sum(),2)}% of matches are ok or good", ""],
      "color": "black",
      "subtitleColor": "black"
    }
)

desc_qual_df = (labelled_evaluation_df.company_description_quality.value_counts()/len(labelled_evaluation_df.query('~company_description_quality.isna()'))*100).reset_index().rename(columns={'index':'desc_quality', 'company_description_quality':'percent'})
desc_qual_chart = alt.Chart(desc_qual_df).mark_bar().encode(
    y=alt.Y('desc_quality:O', title='Description quality', sort='-x'),
    x=alt.X('percent:Q', title='% of job ads'),
    color=alt.condition(alt.datum.desc_quality > 0, alt.value('green'), alt.value('red'))).properties(
    title={
      "text": ["Description Quality"], 
      "subtitle": [f"{round(desc_qual_df.query('desc_quality > 0').percent.sum(),2)}% of company descriptions are ok or good"],
      "color": "black",
      "subtitleColor": "black"
    }
)

sic_method_df = labelled_evaluation_df.sic_method.value_counts().reset_index().rename(columns={'index':'sic_method', 'sic_method':'count'})
sic_method_chart = alt.Chart(sic_method_df).mark_bar().encode(
    y=alt.Y('sic_method:O', title='SIC method', sort='-x'),
    x=alt.X('count:Q', title='count')).properties(
    title={
      "text": ["Count of SIC methods"], 
      "subtitle": [f"closest distance is the most common SIC method in this sample", ""],
      "color": "black",
      "subtitleColor": "black"
    }
)
    
var = "closest distance"
df_has_desc_qual = (labelled_evaluation_df.query(f'sic_method == "{var}"')
                   .dropna(subset=['company_description_quality']))

df_has_desc_qual['company_description_quality_binary'] = np.where(df_has_desc_qual.company_description_quality > 0, 1, 0)

closest_sic = alt.Chart(df_has_desc_qual).mark_circle(size=60).encode(
    x=alt.X('company_description_length', title='Company description length'),
    y=alt.Y('sic_confidence', title='SIC confidence'),
    color=alt.Color('company_description_quality_binary:N', title='Description Quality (Binary)'),
    tooltip=['sic_name', 'sic_confidence', 'match_quality', 'company_description_quality']).properties(title=var)

var = "majority SIC"
maj_sic_qual = (labelled_evaluation_df.query(f'sic_method == "{var}"')
                   .dropna(subset=['company_description_quality']))

maj_sic_qual['company_description_quality_binary'] = np.where(maj_sic_qual.company_description_quality > 0, 1, 0)

maj_sic = alt.Chart(maj_sic_qual).mark_circle(size=60).encode(
    x=alt.X('company_description_length', title='Company description length'),
    y=alt.Y('sic_confidence', title='SIC confidence'),
    color=alt.Color('company_description_quality_binary:N', title='Description Quality (Binary)', legend=None),
    tooltip=['sic_name', 'sic_confidence', 'match_quality', 'company_description_quality']).properties(title=var)

desc_len = (closest_sic | maj_sic).properties(
    title={
      "text": ["Company description length vs. SIC confidence and method"], 
      "subtitle": ["Shorter company description length appear to have been labelled 'bad'.", ""],
      "color": "black",
      "subtitleColor": "black"
    }
)

closet_dist_dens = alt.Chart(df_has_desc_qual).transform_density(
    'sic_confidence',
    as_=['sic_confidence', 'density'],).mark_area().encode(
    x=alt.X("sic_confidence:Q", title="confidence score"),
    y='density:Q',
    color=alt.value('purple'),
).properties(title=f"closest distance")
    
maj_sic_dens = alt.Chart(maj_sic_qual).transform_density(
    'sic_confidence',
    as_=['sic_confidence', 'density'],).mark_area().encode(
    x=alt.X("sic_confidence:Q", title="confidence score"),
    y='density:Q',
    color=alt.value('orange'),
).properties(title=f"majority SIC")
    
dens_plots = (closet_dist_dens | maj_sic_dens).properties(title="SIC confidence density plots")

eval_charts = sic_method_chart & (match_quality | desc_qual_chart) & desc_len 

print('saving charts to html...')
(eval_charts & dens_plots).save(f"{graph_dir}/eval_charts.html")
print('done!')

saving charts to html...
done!
